In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

#### The data

In [2]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [3]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [4]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [5]:
def lstm(X_train, y_train, epochs=500, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [6]:
def scale_data(X):
    scaler = MinMaxScaler()
    X_scaled = X.copy()
    X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
    return X_scaled

In [7]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train_scaled, y_train, epochs=1000)
    y_pred = model.predict(X_test_scaled)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [8]:
rmse_list = []
for i in range(len(latitudes)):
    rmse = cross_validation(final_df, i)
    rmse_list.append(rmse)
    print(f'{device_ids[i]} successful')

Epoch 1/1000
294/294 [==============================] - 4s 6ms/step - loss: 601.5073 - val_loss: 795.4545
Epoch 2/1000
294/294 [==============================] - 1s 3ms/step - loss: 423.9073 - val_loss: 684.3627
Epoch 3/1000
294/294 [==============================] - 1s 3ms/step - loss: 415.4989 - val_loss: 673.4775
Epoch 4/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.1698 - val_loss: 672.3590
Epoch 5/1000
294/294 [==============================] - 1s 3ms/step - loss: 413.4318 - val_loss: 673.0876
Epoch 6/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.9399 - val_loss: 673.3798
Epoch 7/1000
294/294 [==============================] - 1s 3ms/step - loss: 413.8525 - val_loss: 673.6442
Epoch 8/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.2151 - val_loss: 674.8216
Epoch 9/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.7433 - val_loss: 669.7863
Epoch 10/1000
294/294 [=======================

294/294 [==============================] - 1s 3ms/step - loss: 414.3959 - val_loss: 665.6454
Epoch 78/1000
294/294 [==============================] - 1s 3ms/step - loss: 416.3136 - val_loss: 673.0036
Epoch 79/1000
294/294 [==============================] - 1s 3ms/step - loss: 413.7628 - val_loss: 673.7941
Epoch 80/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.0287 - val_loss: 674.4260
Epoch 81/1000
294/294 [==============================] - 1s 3ms/step - loss: 415.4565 - val_loss: 667.0780
Epoch 82/1000
294/294 [==============================] - 1s 3ms/step - loss: 413.1111 - val_loss: 670.3311
Epoch 83/1000
294/294 [==============================] - 1s 3ms/step - loss: 414.4530 - val_loss: 674.8407
Epoch 84/1000
294/294 [==============================] - 1s 3ms/step - loss: 413.8754 - val_loss: 671.7292
Epoch 85/1000
294/294 [==============================] - 1s 3ms/step - loss: 412.9953 - val_loss: 679.9299
Epoch 86/1000
294/294 [============================

294/294 [==============================] - 1s 3ms/step - loss: 405.1768 - val_loss: 638.9005
Epoch 154/1000
294/294 [==============================] - 1s 3ms/step - loss: 406.9081 - val_loss: 670.0742
Epoch 155/1000
294/294 [==============================] - 1s 3ms/step - loss: 405.8206 - val_loss: 687.7623
Epoch 156/1000
294/294 [==============================] - 1s 3ms/step - loss: 406.0991 - val_loss: 670.0366
Epoch 157/1000
294/294 [==============================] - 1s 3ms/step - loss: 407.3676 - val_loss: 656.6253
Epoch 158/1000
294/294 [==============================] - 1s 3ms/step - loss: 405.2503 - val_loss: 689.8468
Epoch 159/1000
294/294 [==============================] - 1s 3ms/step - loss: 406.0525 - val_loss: 667.4049
Epoch 160/1000
294/294 [==============================] - 1s 3ms/step - loss: 406.6240 - val_loss: 671.3464
Epoch 161/1000
294/294 [==============================] - 1s 3ms/step - loss: 404.6088 - val_loss: 659.2173
Epoch 162/1000
294/294 [===================

Epoch 229/1000
294/294 [==============================] - 1s 4ms/step - loss: 401.7977 - val_loss: 703.5436
Epoch 230/1000
294/294 [==============================] - 1s 4ms/step - loss: 402.0452 - val_loss: 671.6721
Epoch 231/1000
294/294 [==============================] - 1s 4ms/step - loss: 400.7414 - val_loss: 700.9289
Epoch 232/1000
294/294 [==============================] - 1s 4ms/step - loss: 401.0325 - val_loss: 679.7305
Epoch 233/1000
294/294 [==============================] - 1s 4ms/step - loss: 402.1256 - val_loss: 657.3698
Epoch 234/1000
294/294 [==============================] - 1s 4ms/step - loss: 399.4717 - val_loss: 656.4634
Epoch 235/1000
294/294 [==============================] - 1s 4ms/step - loss: 401.7874 - val_loss: 687.2733
Epoch 236/1000
294/294 [==============================] - 1s 4ms/step - loss: 400.7027 - val_loss: 669.8853
Epoch 237/1000
294/294 [==============================] - 1s 4ms/step - loss: 401.5352 - val_loss: 647.4568
Epoch 238/1000
294/294 [====

Epoch 305/1000
294/294 [==============================] - 1s 5ms/step - loss: 398.1219 - val_loss: 667.0684
Epoch 306/1000
294/294 [==============================] - 1s 5ms/step - loss: 398.5356 - val_loss: 673.9180
Epoch 307/1000
294/294 [==============================] - 1s 4ms/step - loss: 399.3481 - val_loss: 688.2079
Epoch 308/1000
294/294 [==============================] - 1s 4ms/step - loss: 398.1846 - val_loss: 651.4144
Epoch 309/1000
294/294 [==============================] - 1s 4ms/step - loss: 400.5886 - val_loss: 655.5522
Epoch 310/1000
294/294 [==============================] - 1s 4ms/step - loss: 399.1848 - val_loss: 643.8618
Epoch 311/1000
294/294 [==============================] - 1s 4ms/step - loss: 397.7676 - val_loss: 702.6290
Epoch 312/1000
294/294 [==============================] - 1s 4ms/step - loss: 399.4177 - val_loss: 658.9498
Epoch 313/1000
294/294 [==============================] - 1s 4ms/step - loss: 398.0163 - val_loss: 721.6107
Epoch 314/1000
294/294 [====

...

Epoch 647/1000
320/320 [==============================] - 2s 6ms/step - loss: 372.9706 - val_loss: 379.5382
Epoch 648/1000
320/320 [==============================] - 2s 6ms/step - loss: 373.7634 - val_loss: 370.8427
Epoch 649/1000
320/320 [==============================] - 2s 6ms/step - loss: 372.7785 - val_loss: 370.2201
Epoch 650/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.0232 - val_loss: 373.3380
Epoch 651/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.7082 - val_loss: 369.7610
Epoch 652/1000
320/320 [==============================] - 2s 6ms/step - loss: 372.2519 - val_loss: 368.3101
Epoch 653/1000
320/320 [==============================] - 2s 6ms/step - loss: 373.0356 - val_loss: 370.8661
Epoch 654/1000
320/320 [==============================] - 2s 6ms/step - loss: 372.3906 - val_loss: 371.7001
Epoch 655/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.7611 - val_loss: 372.1617
Epoch 656/1000
320/320 [====

Epoch 723/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.1951 - val_loss: 368.7403
Epoch 724/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.4044 - val_loss: 380.5122
Epoch 725/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.3248 - val_loss: 372.4920
Epoch 726/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.5366 - val_loss: 367.0619
Epoch 727/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.6205 - val_loss: 368.1808
Epoch 728/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.2866 - val_loss: 368.7064
Epoch 729/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.4792 - val_loss: 367.1080
Epoch 730/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.8790 - val_loss: 370.1628
Epoch 731/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.0290 - val_loss: 374.6467
Epoch 732/1000
320/320 [====

Epoch 799/1000
320/320 [==============================] - 2s 6ms/step - loss: 368.9722 - val_loss: 371.6478
Epoch 800/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.3343 - val_loss: 371.5365
Epoch 801/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.1469 - val_loss: 366.7269
Epoch 802/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.7563 - val_loss: 372.5425
Epoch 803/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.6183 - val_loss: 368.5650
Epoch 804/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.6445 - val_loss: 371.1615
Epoch 805/1000
320/320 [==============================] - 2s 6ms/step - loss: 368.5145 - val_loss: 368.7591
Epoch 806/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.8567 - val_loss: 369.6607
Epoch 807/1000
320/320 [==============================] - 2s 6ms/step - loss: 368.1693 - val_loss: 369.2984
Epoch 808/1000
320/320 [====

Epoch 875/1000
320/320 [==============================] - 3s 8ms/step - loss: 373.4893 - val_loss: 373.7721
Epoch 876/1000
320/320 [==============================] - 2s 7ms/step - loss: 373.5913 - val_loss: 370.1280
Epoch 877/1000
320/320 [==============================] - 2s 7ms/step - loss: 372.1921 - val_loss: 372.6346
Epoch 878/1000
320/320 [==============================] - 2s 7ms/step - loss: 371.3281 - val_loss: 366.6660
Epoch 879/1000
320/320 [==============================] - 2s 7ms/step - loss: 373.1528 - val_loss: 369.9573
Epoch 880/1000
320/320 [==============================] - 3s 8ms/step - loss: 375.1682 - val_loss: 370.7649
Epoch 881/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.7985 - val_loss: 371.5739
Epoch 882/1000
320/320 [==============================] - 2s 7ms/step - loss: 372.2183 - val_loss: 377.1540
Epoch 883/1000
320/320 [==============================] - 2s 6ms/step - loss: 374.1900 - val_loss: 376.3242
Epoch 884/1000
320/320 [====

Epoch 951/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.1796 - val_loss: 367.7672
Epoch 952/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.3733 - val_loss: 372.8826
Epoch 953/1000
320/320 [==============================] - 2s 6ms/step - loss: 371.8747 - val_loss: 373.9526
Epoch 954/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.7374 - val_loss: 363.4787
Epoch 955/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.7490 - val_loss: 369.2557
Epoch 956/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.5872 - val_loss: 367.0010
Epoch 957/1000
320/320 [==============================] - 2s 6ms/step - loss: 369.2076 - val_loss: 369.1000
Epoch 958/1000
320/320 [==============================] - 2s 6ms/step - loss: 370.4138 - val_loss: 368.5881
Epoch 959/1000
320/320 [==============================] - 2s 6ms/step - loss: 368.7719 - val_loss: 368.9684
Epoch 960/1000
320/320 [====

In [9]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

20.299085991646823

In [10]:
rmse_list

[12.510381260149892,
 28.323480465515487,
 15.49420247356431,
 21.271306242109105,
 12.94615370923936,
 25.360914528054288,
 20.411129515893364,
 12.926031124400286,
 25.569536699080764,
 28.17772389846135]